# Usage

The basic usage of cmomy is to manipulate central moments.  
We measure two quantities pretend quantities.  The 'energy' and the 'position' of a thing.  We'll construct the average value for each record. Lets say 100 samples each.  

In [1]:
import numpy as np
import xarray as xr
import cmomy 

nsamp = 100
energy = xr.DataArray(np.random.rand(nsamp), dims=['samp'])

position = xr.DataArray(np.random.rand(nsamp, 3), dims=['samp','dim'])

# weight assicated with each sample and each record
w = xr.DataArray(np.random.rand(nsamp), dims=['samp'])

# average over the samples
ce = cmomy.xCentralMoments.from_vals(x=energy, w=w, dim='samp', mom=3)
cp = cmomy.xCentralMoments.from_vals(x=position, w=w, dim='samp', mom=3)

In [2]:
ce

<xCentralMoments(val_shape=(), mom=(3,))>
<xarray.DataArray (mom_0: 4)>
array([5.12327044e+01, 4.51744793e-01, 7.83303173e-02, 2.72363298e-03])
Dimensions without coordinates: mom_0

In [3]:
cp

<xCentralMoments(val_shape=(3,), mom=(3,))>
<xarray.DataArray (dim: 3, mom_0: 4)>
array([[ 5.12327044e+01,  5.24889857e-01,  7.43474632e-02,
        -1.76241600e-03],
       [ 5.12327044e+01,  4.84022188e-01,  7.80664270e-02,
         1.28377807e-03],
       [ 5.12327044e+01,  4.97789444e-01,  8.66146102e-02,
        -3.43961146e-03]])
Dimensions without coordinates: dim, mom_0

## Basic attribues

Notice that there are three `shape` parameters associated with a `CentraMoments` object:
 
* `mom_shape` : shape of the moments.  For single variable, tuple (mom+1,).  For comoments, (mom_0+1, mom_1+1)
* `val_shape`: shape of the 'values' part of the data.  For scalar data, `val_shape = ()`.  For vector data, this is the shape of the passed observation data.
* `shape`: total shape of wrapped moments `shape = val_shape + mom_shape`

In [4]:
ce.mom_shape, ce.val_shape, ce.shape

((4,), (), (4,))

In [5]:
cp.mom_shape, cp.val_shape, cp.shape

((4,), (3,), (3, 4))

To access the underlying data, use the `values` attribute.  `values[...,0]`  is the total weight, `values[...,1]` is the average, and `values[...,k>1]` is the 'kth' central moment.

In [6]:
ce.values

<xarray.DataArray (mom_0: 4)>
array([5.12327044e+01, 4.51744793e-01, 7.83303173e-02, 2.72363298e-03])
Dimensions without coordinates: mom_0

To access all the central moments (zeroth and first included), use

In [7]:
ce.cmom()

<xarray.DataArray (mom_0: 4)>
array([1.        , 0.        , 0.07833032, 0.00272363])
Dimensions without coordinates: mom_0

Likewise, the central moments can be converted to raw moments. 

In [8]:
#<x**k>
ce.rmom()

<xarray.DataArray (mom_0: 4)>
array([1.        , 0.45174479, 0.28240368, 0.20106865])
Dimensions without coordinates: mom_0

Additionally, there are `DataArray` like attributes

In [9]:
ce.coords

Coordinates:
    *empty*

In [10]:
ce.attrs

{}

In [11]:
ce.sizes

Frozen({'mom_0': 4})

In [12]:
ce.dims

('mom_0',)

## Manipulating momements

So we have our averages.  Cool.  Not very special.  But what if instead we repeat our experiment.  Let's say we did the experiment 10 times each time giving a single record.   Then our data would look like

In [13]:
nsamp = 100
nrec = 10
energy = xr.DataArray(np.random.rand(nrec, nsamp), dims=['rec','samp'])
position = xr.DataArray(np.random.rand(nrec, nsamp, 3), dims=['rec','samp','dim'])

# weight assicated with each sample and each record
w = xr.DataArray(np.random.rand(nrec, nsamp), dims=['rec','samp'])

# average over the samples
ce = cmomy.xCentralMoments.from_vals(x=energy, w=w, dim='samp', mom=3)
cp = cmomy.xCentralMoments.from_vals(x=position, w=w, dim='samp', mom=3)

Consider just the energy.  We suspect that there is some correlation between the experiments (they where done in rapid succession).  So we'd like to consider two adjacent experiments as one experiment.  For this, we can use the `block` method.

In [14]:
ce

<xCentralMoments(val_shape=(10,), mom=(3,))>
<xarray.DataArray (rec: 10, mom_0: 4)>
array([[ 4.55274987e+01,  5.07207428e-01,  7.21644574e-02,
        -1.20945060e-03],
       [ 5.47375073e+01,  4.95022017e-01,  7.83430584e-02,
        -4.61254885e-04],
       [ 4.85806445e+01,  4.65745459e-01,  8.41342788e-02,
         4.98907093e-03],
       [ 5.58976270e+01,  5.00885362e-01,  8.73968041e-02,
         2.61331192e-03],
       [ 5.12829347e+01,  4.99215879e-01,  8.08241538e-02,
        -4.20692802e-04],
       [ 4.82695317e+01,  5.48458877e-01,  8.05221427e-02,
        -4.34414104e-03],
       [ 5.53088142e+01,  5.37054088e-01,  7.85813946e-02,
        -3.78878603e-03],
       [ 4.88016717e+01,  5.15384051e-01,  7.60046423e-02,
         2.37179014e-04],
       [ 5.48917004e+01,  5.73157112e-01,  8.36034416e-02,
        -8.54047544e-03],
       [ 4.84993202e+01,  4.74115362e-01,  7.02474413e-02,
         4.49686746e-03]])
Dimensions without coordinates: rec, mom_0

In [15]:
ce.block(block_size=2, dim='rec')

<xCentralMoments(val_shape=(5,), mom=(3,))>
<xarray.DataArray (rec: 5, mom_0: 4)>
array([[ 1.00265006e+02,  5.00555067e-01,  7.55743386e-02,
        -8.56938314e-04],
       [ 1.04478271e+02,  4.84545897e-01,  8.61869736e-02,
         3.80280593e-03],
       [ 9.95524664e+01,  5.23092098e-01,  8.12833819e-02,
        -2.33327747e-03],
       [ 1.04110486e+02,  5.26896283e-01,  7.74904839e-02,
        -1.86006275e-03],
       [ 1.03391021e+02,  5.26697976e-01,  7.97812665e-02,
        -1.45148586e-03]])
Dimensions without coordinates: rec, mom_0

Instead, we can resample the already averaged data.  We produce a 20 new samples taken from the original (averaged) data.

In [16]:
ce.resample_and_reduce(nrep=20, dim='rec')

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


<xCentralMoments(val_shape=(20,), mom=(3,))>
<xarray.DataArray (rep: 20, mom_0: 4)>
array([[ 5.15798453e+02,  5.11524738e-01,  8.28539080e-02,
        -6.83079945e-04],
       [ 5.15282509e+02,  5.24532739e-01,  8.13525895e-02,
        -2.07261593e-03],
       [ 5.02669448e+02,  5.04108968e-01,  8.00886950e-02,
         9.55499800e-04],
       [ 5.05102519e+02,  4.98066069e-01,  7.81837262e-02,
         1.18641305e-03],
       [ 5.00821159e+02,  5.14647993e-01,  7.94192074e-02,
        -7.43089446e-04],
       [ 5.33077772e+02,  5.13237664e-01,  7.97779008e-02,
        -7.84834444e-04],
       [ 5.02065263e+02,  5.13997479e-01,  7.69565763e-02,
        -9.56589629e-04],
       [ 5.14545257e+02,  5.39393426e-01,  7.96072068e-02,
        -3.63840608e-03],
       [ 5.33011513e+02,  5.10143994e-01,  8.23302308e-02,
        -2.33841939e-04],
       [ 5.11125917e+02,  5.27194776e-01,  8.04930943e-02,
        -2.61028797e-03],
       [ 5.18697119e+02,  5.19241142e-01,  8.14099775e-02,
       

This is differnt than the usual 'resample values'.  This is also available if the original data is available.

In [17]:
a = energy.stack(c=['rec','samp'])

In [18]:
b = w.stack(c=['rec','samp'])

In [19]:
# consider 'all' the data for this
out = cmomy.xCentralMoments.from_resample_vals(energy.stack(c=['rec','samp']), w=w.stack(c=['rec','samp']), dim='c', nrep=20, mom=3)

We can also reduce our original data across all the records

In [20]:
ce.reduce(dim='rec')

<xCentralMoments(val_shape=(), mom=(3,))>
<xarray.DataArray (mom_0: 4)>
array([ 5.11797250e+02,  5.12310397e-01,  8.03830920e-02, -6.11039124e-04])
Dimensions without coordinates: mom_0